# Data Exploration

# 1 - Information

In [ ]:
# Author: Pierre Oreistein
# Description inspired by: https://towardsdatascience.com/predictive-maintenance-of-turbofan-engines-ec54a083127

# 2 - Packages

In [ ]:
%reload_kedro

In [ ]:
# Math Packages
import numpy as np

# Statistical packages
from scipy.stats import exponweib, weibull_max, weibull_min

# Graphics Packages
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# Data Handling Packages
import pandas as pd

# Machine Learning Packages
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Prevent unecessary warnings
from warnings import filterwarnings

filterwarnings("ignore", ".*`should_run_async`.*")

# 3 - Data Exploration

## 3.1 - Load Data

 The engines operate normally in the beginning but develop a fault over time. For the training sets, the engines are run to failure, while in the test sets the time series end ‘sometime’ before failure. The goal is to predict the Remaining Useful Life (RUL) of each turbofan engine.

Datasets include simulations of multiple turbofan engines over time, each row contains the following information:
1. Engine unit number
2. Time, in cycles
3. Three operational settings
4. 21 sensor readings

In [ ]:
X_train_df = catalog.load("X_train_raw")

# Display train_df
X_train_df.head()

In [ ]:
X_test_df = catalog.load("X_test_raw")

# Display train_df
X_test_df.head()

In [ ]:
y_test_df = catalog.load("y_test_raw")

# Display train_df
y_test_df.head()

## 3.2 - Check Data Quality

In [ ]:
# Check the operating mode. We should only observe 1
X_train_df[["setting_1", "setting_2", "setting_3"]].describe()

Looking at the standard deviations of settings 1 and 2, they aren’t completely stable. The fluctuations are so small however, that no other operating conditions can be identified. As there is only one operating mode, we will discard these three features in our final training set.

In [ ]:
# Describe sensors
SENSORS_NAME_L = [f"s_{i}" for i in range(20)]

# Check the sensors data (variablity mainly)
X_train_df[SENSORS_NAME_L].describe().transpose()

By looking at the standard deviation it’s clear sensors 1, 10, 18 and 19 do not fluctuate at all, these can be safely discarded as they hold no useful information. Inspecting the quantiles indicates sensors 5, 6 and 16 have little fluctuation and require further inspection. Sensors 9 and 14 have the highest fluctuation, however this does not mean the other sensors can’t hold valuable information.

# 4 - Feature Engineering

In [ ]:
def add_remaining_useful_life(df):
    """Add remaining useful life to df."""
    # Get the total number of cycles for each unit
    grouped_by_unit_df = df.groupby(by="unit_nb")
    max_cycle_s = grouped_by_unit_df["time"].max()

    # Merge the max cycle back into the original frame
    result_df = df.merge(
        max_cycle_s.to_frame(name="max_cycle"), left_on="unit_nb", right_index=True
    )

    # Calculate remaining useful life for each row
    remaining_useful_life_s = result_df["max_cycle"] - result_df["time"]
    result_df["RUL"] = remaining_useful_life_s

    # drop max_cycle as it's no longer needed
    result_df = result_df.drop("max_cycle", axis=1)
    return result_df

In [ ]:
# Add remaining useful life
train_df = add_remaining_useful_life(X_train_df)

# Display Resulting DataFrame
train_df.head()

# 5 - Data Visualisation

## 5.1 - Visualise Sensors

In [ ]:
def plot_sensor(df: pd.DataFrame, sensor_name: str, scaling: str = False) -> None:
    """Plot the sensor value over time for the different units/engines."""
    # Initialisation of the figure
    plt.figure(figsize=(13, 5))

    # Subsample some units for faster display
    grouped_df = df.groupby("unit_nb")
    for unit, g_df in grouped_df:
        if unit % 10 == 0:  # only plot every 10th unit_nr

            # Scaling of the data
            if scaling:

                scaler = StandardScaler()
                g_df[sensor_name] = scaler.fit_transform(
                    g_df[sensor_name].values.reshape(-1, 1)
                )

            plt.plot("RUL", sensor_name, data=g_df)

    # Reverse the x-axis so RUL counts down to zero
    plt.xlim(275, 0)

    # Customize axis and title
    plt.xticks(np.arange(0, 275, 25))
    plt.ylabel(sensor_name)
    plt.xlabel("Remaining Use fulLife")
    plt.show()

### 5.1.1 - Discarded sensors

In [ ]:
# Plot the different sensor time series
for sensor_name in ["s_0", "s_4", "s_9", "s_15", "s_17", "s_18"]:

    print(f"----- Sensor {sensor_name} -----")
    plot_sensor(df=train_df, sensor_name=sensor_name)
    print("\n")

The graph of sensors 0, 4, 9, 15, 17 and 18 looks similar, the flat line indicates the sensors hold no useful information, which reconfirms our conclusion from the descriptive statistics. 

In [ ]:
# Plot the different sensor time series
for sensor_name in ["s_0", "s_4", "s_9", "s_15", "s_17", "s_18"]:

    print(f"----- Sensor {sensor_name} -----")
    plot_sensor(df=train_df, sensor_name=sensor_name, scaling=True)
    print("\n")

Even with scaling, the sensors 0, 4, 9, 15, 17 and 18 does not seem to contain useful information. We will discard them for the predictions.

In [ ]:
# Plot the different sensor time series
for sensor_name in ["s_5"]:

    print(f"----- Sensor {sensor_name} -----")
    plot_sensor(df=train_df, sensor_name=sensor_name, scaling=False)
    print("\n")

Sensor readings of sensor 6 peak downwards at times but there doesn’t seem to be a clear relation to the decreasing RUL. We will remove this sensor from our final training set.

### 5.1.2 - Sensors preserved

In [ ]:
# Plot the different sensor time series
for sensor_name in ["s_1", "s_2", "s_3", "s_7", "s_10", "s_12", "s_14", "s_16"]:

    print(f"----- Sensor {sensor_name} -----")
    plot_sensor(df=train_df, sensor_name=sensor_name, scaling=True)
    print("\n")

Sensor 1 shows a rising trend, a similar pattern can be seen for sensors 1, 2, 3, 7, 10, 12, 14 and 16. We will keep these features in our training set.

In [ ]:
# Plot the different sensor time series
for sensor_name in ["s_6", "s_11", "s_19"]:

    print(f"----- Sensor {sensor_name} -----")
    plot_sensor(df=train_df, sensor_name=sensor_name, scaling=True)
    print("\n")

Sensor 6 shows a declining trend, which can also be seen in sensors 12, 19 and 11. We will keep these sensors in our final training set.

In [ ]:
# Plot the different sensor time series
for sensor_name in ["s_8"]:

    print(f"----- Sensor {sensor_name} -----")
    plot_sensor(df=train_df, sensor_name=sensor_name, scaling=True)
    print("\n")

Sensor 8 has a similar pattern as sensor 13. We will keep its information for our final training set.

## 5.2 - Traget Visualisation

In [ ]:
# Display the Remaining Useful Life Histogram of our training set
max_rul_df = train_df[["unit_nb", "RUL"]].groupby("unit_nb").max().reset_index()
max_rul_df["RUL"].hist(bins=10, figsize=(15, 7))

plt.xlabel("RUL")
plt.ylabel("frequency")
plt.show()

Let's compare this distribution to a Weibull distribution

In [ ]:
# First, let's rescale the rul
rul_a = max_rul_df["RUL"].values.reshape(-1, 1)
scaler = MinMaxScaler()
rul_a = scaler.fit_transform(rul_a)

# Let's display the best fitted Weibull distribution vs the RUL observed
plt.figure(figsize=(15, 7))
plt.hist(rul_a, bins=10, density=True, alpha=0.8)
plt.scatter(
    rul_a, weibull_max.pdf(rul_a, *weibull_max.fit(rul_a)), c="red", zorder=10
)
plt.show()

We can notive that the best Weibull fit well the dataset. It is therefore legitimate to try to fit a Weibull distribution on our dataset.